In [ ]:
import pandas as pd
import joblib
import os
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, f1_score, recall_score, precision_score

FILENAME = 'minesweeper_dataset.csv'
MODEL_NAME = 'minesweeper_ai_model.pkl'

# 1. Caricamento dataset
if not os.path.exists(FILENAME):
    print(f"ERRORE: Il file '{FILENAME}' non esiste.")
    exit()

print(f"Caricamento dataset: {FILENAME}...")
df = pd.read_csv(FILENAME)

print(f"Righe totali caricate: {len(df)}")

# Rimozione duplicati
df = df.drop_duplicates()
print(f"Righe uniche (dataset effettivo): {len(df)}")

# 2. Preparazione dati Input (X) e Output (y)
X = df.drop('safe', axis=1)
y = df['safe']

# 3. Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 4. Configurazione Random Forest
# Manteniamo questi limiti per avere un modello intelligente e leggero
model = RandomForestClassifier(
    n_estimators=100,       #Max 100 alberi
    max_depth=20,           #Profondità max
    min_samples_leaf=50,    #Ignora il rumore statistico (regole con <50 esempi)
    max_features='sqrt',
    n_jobs=-1,              
    random_state=42,
    verbose=1
)

print("Addestramento in corso...")
model.fit(X_train, y_train)

# 5. Valuta
print("\nCalcolo metriche...")
predictions = model.predict(X_test)

# Calcolo metriche modello
acc = accuracy_score(y_test, predictions)
prec = precision_score(y_test, predictions, average='weighted')
rec = recall_score(y_test, predictions, average='weighted')
f1 = f1_score(y_test, predictions, average='weighted')

print("-" * 40)
print(f"RISULTATI DEL MODELLO")
print("-" * 40)
print(f"ACCURACY  (Precisione globale): {acc * 100:.2f}%")
print(f"RECALL    (Capacità di trovare): {rec * 100:.2f}%")
print(f"F1-SCORE  (Bilanciamento):      {f1 * 100:.2f}%")
print("-" * 40)

print("\nReport Dettagliato per Classe:")
# 0 = Mina (Non Safe), 1 = Sicuro (Safe)
print(classification_report(y_test, predictions))

# 6. Salva
print(f"Salvataggio modello '{MODEL_NAME}'...")

joblib.dump(model, MODEL_NAME) 